In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads
        assert self.head_dim * num_heads == embed_size, "Embed size needs to be divisible by heads"
        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(query)

        values = values.view(N, value_len, self.num_heads, self.head_dim).transpose(1, 2)
        keys = keys.view(N, key_len, self.num_heads, self.head_dim).transpose(1, 2)
        queries = queries.view(N, query_len, self.num_heads, self.head_dim).transpose(1, 2)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / math.sqrt(self.head_dim), dim=3)

        out = torch.einsum("nhqk,nkhd->nqhd", [attention, values]).contiguous()
        out = out.view(N, query_len, self.num_heads * self.head_dim)

        return self.fc_out(out)


In [ ]:
class FeedForward(nn.Module):
    def __init__(self, embed_size, hidden_size, dropout):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(embed_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, embed_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        return self.fc2(x)


In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, num_heads, ff_hidden_size, dropout):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(embed_size, num_heads)
        self.ff = FeedForward(embed_size, ff_hidden_size, dropout)
        self.layernorm1 = nn.LayerNorm(embed_size)
        self.layernorm2 = nn.LayerNorm(embed_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        x = self.layernorm1(query + self.dropout(attention))
        forward = self.ff(x)
        return self.layernorm2(x + self.dropout(forward))


TypeError: 'list' object is not callable